# Extracting Data from a PDF Table: AI Ethics Policies
# Part 2: The State of the Art of PDF Table Extraction

_Alistair Boyer_

Close your eyes and imagine a world where every dataset is nicely curated. 
There is a detailed data dictionary that explains the purpose of each column and all the values that could ever be contained within.
Each column has an optimal type associated with the data that provides convenient methods to access the various properties of the data. 
There are no duplicates across rows. There are no ambiguity across values. 
Now, __wake up__ because we all know that this is not the world we live in.
Working with data most often involves spending a disproportionate amount of time extracting and cleaning data from frustrating sources. 


A clear contender for the worst offender in the category of swallowing valuable time is tabular data stored in PDF files.
These portable files that are super convenient for presenting formatted information and dominate the document landscape.
In 2015, [Adobe's Phil Ydens revealed](https://youtu.be/5Axw6OGPYHw) that >70 million new PDFs were saved every day in the Google Cloud; and simple maths extrapolates the total number of PDFs to be well over 2 trillion. 
However, all this portability comes at a heavy cost. The data within a PDF file is stored as an almost random selection of texts and shapes and their properties and coordinates within each page. 
We have all experienced the actuality of this when dragging across text within a PDF page to find various words and phrases selected in a random order including page number and headers.
What makes extracting data from a PDF even more frustrating is the knowledge that this exact same data exists somewhere in a much more convenient form that someone has decided not to share. 

I have been reviewing AI Ethics policies and came across the 2019 [perspective article](https://doi.org/10.1038/s42256-019-0088-2) in _Nature Machine Intelligence_ by Anna Jobin, Marcello Ienca and Effy Vayena, titled "The Global Landscape of AI Ethics Guidelines" [__2019__, _Vol 1_, pp 389-399].
The article covers principles and guidelines for ethical artificial intelligence published across the globe and includes a list of these policies as Table S2 within the [PDF Supplementary Information](https://www.nature.com/articles/s42256-019-0088-2#Sec17). 


This two-part article describes my elongated journey into this challenge and covers the state-of-the-art for PDF extraction in May 2024.
[__Part 1__](RateLimiting_CompaniesHouse_Part1.ipynb) starts with a custom approach to table extraction using __pypdf__ to match all text within the rectangles that make up the table and building an automated response to getting the data.
[__Part 2__](RateLimiting_CompaniesHouse_Part2.ipynb) looks at the python packages that are designed for tabular data extraction, starting with __Adobe__'s API for table extraction that reinforces their PDF experience with ML and AI techniques.
Then looking at a smorgasbord of python packages: __pdfplumber__, __tabula__ and __camelot__.

In [2]:
import re
import io
import pandas

In [3]:
import requests
import pathlib

# The SI data is available from the publisher; and is saved locally to GlobalLandscapeOfAIEthicsSI.pdf

PDF_URL = "https://static-content.springer.com/esm/art%3A10.1038%2Fs42256-019-0088-2/MediaObjects/42256_2019_88_MOESM1_ESM.pdf"

PDF = pathlib.Path("data/GlobalLandscapeOfAIEthicsSI.pdf")

if not PDF.is_file():
    with PDF.open("wb") as f:
        f.write(requests.get(PDF_URL).content)

## PDF Data

A PDF file is a collection of text and graphics information and the information required to present this on a page, _i.e._ location, font, transformation, kerning, etc&hellip;.
A significant challenge is that unlike something like HTML where the structure of the file reflects the structure of the document, in PDF there is __no guarantee__ that order of data is correlated with its location on a page. 
There is an excellent description of PDF text operations in this [free book chapter by _Ryan Hodson_](https://www.syncfusion.com/succinctly-free-ebooks/pdf/text-operators)
and a handy visual cheat-sheet of PDF operators created [by the PDF Association](https://pdfa.org/wp-content/uploads/2023/08/PDF-Operators-CheatSheet.pdf).


However, you don't have to wade through the complex PDF file by yourself and there are many packages that have been created to streamline this process. These packages use various strategies for table extraction including: the same line/rectangle finding strategy as above; to grouping text by position alone; and AI and machine learning techniques.

## pypdf - A Custom Approach

`pip install pypdf`

In [__Part 1__](RateLimiting_CompaniesHouse_Part1.ipynb) of this article, a custom approach was built using `pypdf`.

As part of the data extraction process, these functions were created that are useful in general for this dataset so have been repeated here:
- `column_labels_from_first_row()` sets the DataFrame column labels from the first row and then filters out that row.
- The table data is split across several pages and this produces fragmentary output that caa be recombined using `reconstruct_split_rows()`.

In [6]:
def column_labels_from_first_row(df):
    
    # set the column labels from the row 0
    df.columns = df.iloc[0]
    
    # filter out row 0
    df = df.iloc[1:]
    
    # reset the index
    df.reset_index(drop=True, inplace=True)
    
    return df


def row_has_nans(df):
    return df.isna().any(axis=1)


def reconstruct_split_rows(df, split_row_filter=row_has_nans):

    # calculate a row_id 
    row_id = (~split_row_filter(df)).cumsum()
    
    return (
        df
        # fill any nulls with an empty string
        .fillna("")
        # group by the row id
        .groupby(row_id)
        # aggregate the data by joining the strings
        .agg(lambda x: "".join(x))
        # reset the index
        .reset_index(drop=True)
    )

Overall, using `pypdf` was a complex process but delivered a perfect representation of the tabular data.

In [131]:
import pandas

# load part 1 form a csv file
df_pypdf = pandas.read_csv(PDF.with_suffix(".csv"))

df_pypdf.head()

,Name of Document/Website,Name of guidelines/principles,Issuer,Country of issuer,Type of issuer,Date of publishing,Target audience,Retrieval
0,Principles of robotics,"Principles for designers, builders and users o...",Engineering and Physical Sciences Research Cou...,UK,Science foundation,1-Apr-2011,"multiple (public, developers)",Linkhubs
1,Ethique de la recherche en robotique,Préconisations,CERNA (Allistene),France,Research alliance,xx-Nov-2014,researchers,Citation chaining
2,Unified Ethical Frame for Big Data Analysis. I...,Values for an Ethical Frame,The Information Accountability Foundation,USA,NPO/Charity,xx-Mar-2015,unspecified,Citation chaining
3,Ethics Policy,IIIM's Ethics Policy,Icelandic Institute for Intelligent Machines (...,Iceland,Academic and research institution,31-Aug-2015,self,Linkhubs
4,The AI Now Report. The Social and Economic Imp...,Key recommendations,AI Now Institute,USA,Academic and research institution,22-Sep-2016,unspecified,Citation chaining


## Adobe

`pip install pdfservices-sdk`

Adobe may have gifted the PDF format to the world but _"what the right hand gives, the left hand takes"_ and Adobe kept the format proprietary until 2008, controlling many of the most important PDF features including editing and manipulation. 
Today, Adobe offers a powerful toolchain for working with PDFs but this is accessed through an API that is usage tracked against user accounts.
Adobe's API tools include document hosting (free unlimited); document generation; document signing; and (most-relevant-here) text-extraction.
The text extract API combines Adobe's extensive experience with the PDF format with machine learning and natural language processing techniques.
The API has python bindings but the text extract API is capped at __500 requests per month__ for the free tier.

The __API setup__ is helpfully described online in a [quickstart guide](https://developer.adobe.com/document-services/docs/overview/pdf-extract-api/quickstarts/python/). 
Once registering you account with Adobe, navigating to https://developer.adobe.com/document-services yields a prominent __Get credentials__ button on the menu.
Clicking this opens up a new site with a choice between PDF Services API (what we need here) and PDF Embed API (free unlimited for hosting PDF).
Clicking create credentials on PDF Services API navigates to a page where you can specify the name for your credentials and select your language of choice. 
Selecting python, there is a checkbox option to generate language-specific code samples that demonstrate how the API can be used.
N.B. Your authentication credentials are supplied to you in a `.json` file - this should be protected as sensitive information and accessed using e.g. environment variables.
Your project credentials and usage statistics are also available in the [project dashboard](https://developer.adobe.com/console/projects).


This code is derived from the personalised code sample that provided upon setting up the application at Adobe.
The first thing to note is that Adobe's API relies heavily on chaining methods, often found in e.g. Java. 

- To protect against using all my allowance, the code here first checks if the result has already been calculated. The result is returned from Adobe as a zip file that I have chose to save as the same name as the original PDF file but with the `.zip` suffix.  
- To interface Adobe's API, the relevant credentials need to me exchanged. `credentials` is created as an instance of `Credentials` using the builder `.service_principal_credentials_builder()`, with the appropriate information supplied using the `.with_client_id()` and `.with_client_secret()` chained methods, before being `.built()`. The secret credentials are protected by storing as environment variables that can be retrieved using the [`dotenv` package](https://pypi.org/project/python-dotenv/). 
- An `ExecutionContext`: `context` is created from the `credentials`.
- The desired `operation` is initialised using `ExtractPDFOperation.create_new()`. 
- The local path of the file is supplied to the operation by creating `source` as a `FileRef` object describing the local PDF file's full path provided from the `.resolve()` of the `pathlib.Path` object. - The `source` is then registered against the `operation` using the `.set_source()` method.
- The final stage of the setup is to describe the operation type in `options`. This is an `ExtractPDFOptions` object created using the `.builder()` method with the desired text element type: `ExtractElementType.TEXT` and `ExtractElementType.TABLES` added using the `.with_element_to_extract()` chained method, before being `.built()`.
- The options are registered against the `operation` using the `.set_options()` method.
- The result is generated form the operation and context `operation.execute(context)` and this is saved locally `.save_as()` as a `.zip` file.

In [10]:
import dotenv
import json

from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType

ZIP = PDF.with_suffix(".zip")

if not ZIP.is_file():

    # setup access credentials
    credentials = (
        # base object and builder
        Credentials.service_principal_credentials_builder()
        # register user data
        .with_client_id(dotenv.dotenv_values()['adobe_client_id'])
        .with_client_secret(dotenv.dotenv_values()['adobe_client_secret'])
        # build
        .build()
    )

    # set the context
    context = ExecutionContext.create(credentials)

    # choose the operation
    operation = ExtractPDFOperation.create_new()

    # choose the file source
    source = FileRef.create_from_local_file(PDF.resolve())
    operation.set_input(source)

    # use optons to select the element type to extract
    options = (
        # base object and builder
        ExtractPDFOptions.builder()
        # select text and tables to extract
        .with_element_to_extract(ExtractElementType.TEXT)
        .with_element_to_extract(ExtractElementType.TABLES)
        # build
        .build()
    )
    operation.set_options(options)

    # get result
    result = operation.execute(context)

    # save result ouput
    result.save_as(ZIP)

The result `.zip` file contains a `.json` file called `'structuredData.json'` that holds the result of the extraction.
This is a large `.json` file contains details of all the elements within the PDF referenced by the `"elements"` key. 
Adobe parsed the PDF to include a document path for each element, for example: `"//Document/Table/TR/TD/P"`.
This is similar to a xml XPath or the HTML location of the element if the data were presented as a website. 

However, Adobe was tasked with extracting with the `ExtractElementType.TABLES` option too. 
Within the result `.zip` file there is a folder `tables/` containing all the tables that Adobe could construct, i.e. it did all the hard work for us.
The relevant table is `tables/fileoutpart1.xlsx` and this can be loaded into a DataFrame using `pandas.read_excel()`
Again, the split data needs recombining. There are also many `_x000D_` that have been added as a byproduct of excel creation.
These can be removed by applying the `openpyxl.utils.escape.unescape` function. 

In [12]:
import zipfile
import openpyxl

with zipfile.ZipFile(ZIP, 'r') as z:
    df_adobe_auto = pandas.read_excel(io.BytesIO(z.read('tables/fileoutpart1.xlsx')))

# reconstruct split rows    
df_adobe_auto = reconstruct_split_rows(df_adobe_auto)
    
# remove import remnants
df_adobe_auto.columns = pandas.Series(df_adobe_auto.columns).apply(openpyxl.utils.escape.unescape).str.strip()
for label in df_adobe_auto:
    df_adobe_auto[label] = df_adobe_auto[label].astype(str).apply(openpyxl.utils.escape.unescape).str.strip()
    
# look at the first 5 rows
df_adobe_auto.head(5)

,Name of Document/Website,Name of guidelines/principl es,Issuer,Country of issuer,Type of issuer,Date of publishi ng,Target audience,Retrieval
0,Principles of robotics,"Principles for designers, builders and users o...",Engineering and Physical Sciences Research Cou...,UK,Science foundation,1-Apr-2011,"multiple (public, developers)",Linkhubs
1,Ethique de la recherche en robotique,Préconisations,CERNA (Allistene),France,Research alliance,xx-Nov-2014,researchers,Citation chaining
2,Unified Ethical Frame for Big Data Analysis. I...,Values for an Ethical Frame,The Information Accountability Foundation,USA,NPO/Charity,xx-Mar-2015,unspecified,Citation chaining
3,Ethics Policy,IIIM's Ethics Policy,Icelandic Institute for Intelligent Machines \...,Iceland,Academic and research institution,31-Aug-2015,self,Linkhubs
4,The AI Now Report. The Social and Economic Imp...,Key recommendations,AI Now Institute,USA,Academic and research institution,22-Sep-2016,unspecified,Citation chaining


## pdfplumber

`pip install pdfplumber`

pdfplumber is a [python package](https://pypi.org/project/pdfplumber/) that captures all text characters from within a PDF with detailed information about their layout along with any rectangles and lines on the page. 
pdfplumber is able to use all this information to extract tabular data.
pdfplumber is built upon the the [`pdfminer` package](https://pypi.org/project/pdfminer/) and is __pure python__.

The process for extracting our tabular data is very streamlined in this dedicated package:
- A pdfplumber `pdf.PDF` object is opened using the `.open()` method with the path of the file, then the target pages `[3:8]` are accessible by indexing the `.pages` property.
- Each page has an `.extract_table()` method that produces tabular data as a list that can be converted into a `pandas.DataFrame` and collected in a list `pdfplumber_tables`.
- All the information can be combined into a single DataFrame using `pandas.concat()`.
- The column labels are collected from the first row using the `column_labels_from_first_row()` function.
- Data that was split across page breaks is recombined using the `reconstruct_split_rows()` function. Here, the null data is not `numpy.nan` but empty `str` so needs to be found using a lambda function that returns the number of items having `.len()` of `0`: `.apply(lambda row: (row.str.len() == 0).sum(), axis=1)`.

In [207]:
import pdfplumber

# initialise a list to collect tables
pdfplumber_tables = list()

# get the tables from each page 3:8
for page in pdfplumber.open(PDF).pages[3:8]:
    df = pandas.DataFrame(page.extract_table())
    pdfplumber_tables.append(df)

# concat the dataframes on each page into one dataframe
df_pdfplumber = pandas.concat(pdfplumber_tables)

# strip the leading and trailing spaces
df_pdfplumber = df_pdfplumber.apply(lambda series: series.str.strip())

# convert new lines to spaces
df_pdfplumber = df_pdfplumber.replace(r"\n", " ", regex=True)

# set the headings to the first row
df_pdfplumber = column_labels_from_first_row(df_pdfplumber)

# recombine split rows
df_pdfplumber = reconstruct_split_rows(df_pdfplumber,  lambda df: df.apply(lambda row: (row.str.len() == 0).sum(), axis=1) > 3)

# look at the top 5 rows
df_pdfplumber.head(5)

,Name of Document/Website,Name of guidelines/principl es,Issuer,Country of issuer,Type of issuer,Date of publishi ng,Target audience,Retrieval
0,Principles of robotics,"Principles for designers, builders and users o...",Engineering and Physical Sciences Research Cou...,UK,Science foundation,1-Apr- 2011,"multiple (public, developers)",Linkhubs
1,Ethique de la recherche en robotique,Préconisations,CERNA (Allistene),France,Research alliance,xx-Nov- 2014,researchers,Citation chaining
2,Unified Ethical Frame for Big Data Analysis. I...,Values for an Ethical Frame,The Information Accountability Foundation,USA,NPO/Charity,xx-Mar- 2015,unspecified,Citation chaining
3,Ethics Policy,IIIM's Ethics Policy,Icelandic Institute for Intelligent Machines (...,Iceland,Academic and research institution,31-Aug- 2015,self,Linkhubs
4,The AI Now Report. The Social and Economic Imp...,Key recommendations,AI Now Institute,USA,Academic and research institution,22-Sep- 2016,unspecified,Citation chaining


## tabula

`pip install tabula-py` 

The [tabula-py python package](https://pypi.org/project/tabula-py/) is a python wrapper for [tabula java](https://github.com/tabulapdf/tabula-java).
tabula is a set of tools for extracting tabular data from a PDF file and has been implemented as [installable software](https://tabula.technology/).
Using tabula in python requires access to a __java__ installation. For me, Google colab resolved this automagically but I had some difficulty locally, experiencing inconsistent errors on import and having to manually modify system environment variables so that the package could find my java installation. 

Once everything was set up, the process for extracting our tabular data in tabula is simple:
- tabula's `.read_pdf()` function creates the PDF object. 
The `pages=` argument targets our desired data;
the encoding `encoding=` can be supplied as `"windows-1252"`;
and `multiple_tables=` should be `False` because Table S2 is a single table spread across multiple pages.
- The output of this function is a list of `pandas.DataFrame` of length 1 because `multiple_tables=` was `False`. 
- The column labels were already found correctly.
- Other outputs had information split across 3 extra rows where the PDF had page breaks but this data is much more fragmented with __334__ total rows. However, this data can be combined in exactly the same way using `reconstruct_split_rows()`.
- All the multi-line was separated by a `"\r"` line break - these could be replaced with spaces using the appropriate `.replace()` methods.



In [211]:
import tabula

# load the tables from the target page range
tabula_tables = tabula.read_pdf(PDF, pages="4-8", encoding="windows-1252", multiple_tables=False)

# recombine split rows
df_tabula = reconstruct_split_rows(tabula_tables[0])

# remove \r from data
df_tabula.columns = map(lambda s:s.replace('\r', ' '), df_tabula.columns)
df_tabula = df_tabula.replace(r'\r', '', regex=True)

# inspect the top 5 rows
df_tabula.head(5)

,Name of Document/Website,Name of guidelines/principl es,Issuer,Country of issuer,Type of issuer,Date of publishi ng,Target audience,Retrieval
0,Principles of robotics,"Principles fordesigners, buildersand users of ...",Engineering andPhysical SciencesResearch Counc...,UK,Sciencefoundation,1-Apr-2011,"multiple (public,developers)",Linkhubs
1,Ethique de larecherche en robotique,Préconisations,CERNA (Allistene),France,Research alliance,xx-Nov-2014,researchers,Citationchaining
2,Unified Ethical Framefor Big Data Analysis.IAF...,Values for anEthical Frame,The InformationAccountabilityFoundation,USA,NPO/Charity,xx-Mar-2015,unspecified,Citationchaining
3,Ethics Policy,IIIM's Ethics Policy,Icelandic Institute forIntelligent Machines(IIIM),Iceland,Academic andresearchinstitution,31-Aug-2015,self,Linkhubs
4,The AI Now Report.The Social andEconomicImplic...,Keyrecommendations,AI Now Institute,USA,Academic andresearchinstitution,22-Sep-2016,unspecified,Citationchaining


## camelot

`pip install camelot-py`

[camelot](https://camelot-py.readthedocs.io/) is another [package](https://pypi.org/project/camelot-py/) that can be used to extract tabular PDF data and it has a suitably named accompanying web interface [excalibur](https://excalibur-py.readthedocs.io/). 
The camelot name comes from the [code name of the project](https://en.wikipedia.org/wiki/History_of_PDF) that was the progenitor of PDF itself.
The main [advantage](https://camelot-py.readthedocs.io/en/master/user/intro.html#why-another-pdf-table-extraction-library) of camelot over other PDF extraction packages is that it provides easy access to metadata and fine-tuning that can help when extraction is not straightforward. camelot requires __ghostscript__ and __Tkinter__ to be installed on your system and this can be a bit of a fiddle to set up depending on your environment.
camelot is built upon the the [`pdfminer` package](https://pypi.org/project/pdfminer/).


The process for extracting our tabular data in camelot is:
- camelot's `.read_pdf()` function creates the PDF object using the `pages=` argument to target our desired data; and `strip_text="\n"` so that multi-line data is joined without including the newline character.
- Separate DataFrame objects are accessible from each table's `.df` property and these can be combined using `pandas.concat()`.
- The column labels are collected from the first row.
- Data that was split across page breaks is recombined.

In [203]:
import camelot

# load tables from pages 4-8
camelot_tables = camelot.read_pdf(str(PDF), pages='4-8', strip_text="\n")

# concat each page's df into one dataframe
df_camelot = pandas.concat(table.df for table in camelot_tables)

# get column labels from first row
df_camelot = column_labels_from_first_row(df_camelot)

# recombine split rows
df_camelot = reconstruct_split_rows(df_camelot, lambda df: df.apply(lambda row: (row.str.len() == 0).sum(), axis=1) > 3)

# look at the first 5 rows
df_camelot.head(5)

,ame of Document/Website,Name of guidelines/principles,Issuer,Country of issuer,Type of issuer,Date of publishing,Target audience,Retrieval
0,Principles of robotics,"Principles for designers, builders and users o...",Engineering and Physical Sciences Research Cou...,UK,Science foundation,1-Apr-2011,"multiple (public, developers)",Linkhubs
1,Ethique de la recherche en robotique,Préconisations,CERNA (Allistene),France,Research alliance,xx-Nov-2014,researchers,Citation chaining
2,Unified Ethical Frame for Big Data Analysis. I...,Values for an Ethical Frame,The Information Accountability Foundation,USA,NPO/Charity,xx-Mar-2015,unspecified,Citation chaining
3,Ethics Policy,IIIM's Ethics Policy,Icelandic Institute for Intelligent Machines (...,Iceland,Academic and research institution,31-Aug-2015,self,Linkhubs
4,The AI Now Report. The Social and Economic Imp...,Key recommendations,AI Now Institute,USA,Academic and research institution,22-Sep-2016,unspecified,Citation chaining


## pdfminer, PyMuPDF and pdfquery
These are other popular PDF packages for python but were beyond the scope of this document because I couldn't find a fast way to extract quality tabular data using them.
Feel free to correct me and I will update this document!

`pip install pdfminer`
- [`pdfminer` is a powerful package](https://pypi.org/project/pdfminer/) that `pdfplumber` and `camelot` are both built upon.


`pip install PyMuPDF`
- [`PyMuPDF` is a popular PDF extraction](https://pypi.org/project/PyMuPDF/) tool but I couldn't shape it into getting valuable data without having to invest significant efforts.

`pip install pdfquery`
- [`pdfquery` is a package](https://pypi.org/project/pdfquery/) that creates an XPath-type tree for the PDF document allowing for powerful queries to be executed on the document.


## OCR
When extracting text from a PDF there is a further complication that, in addition to fragmentary text vector data littering the page, text can be stored as a rasterised images within the PDF.
This results in an oft-used approach to extracting PDF data is processing the whole PDF as an image and then extracting relevant data using OCR (optical character recognition)!
`pytesseract` is a popular [python OCR package](https://pypi.org/project/pytesseract/) if you face this challenge and there is a plethora of online offerings.

# Comparison

The quality of the output from `pypdf` was checked manually against the PDF data source and was found to be 100% accurate. 
To measure the performance of the other libraries the [Levenshtein edit distance](https://en.wikipedia.org/wiki/Levenshtein_distance) can be calculated against the pypdf output `df_pypdf`.
The [natural language toolkit NLTK](https://www.nltk.org/) has tools for [measuring the distance](https://www.nltk.org/api/nltk.metrics.distance.html#module-nltk.metrics.distance) between two strings.

Each of the data sources is gathered into a dictionary.

In [72]:
outputs = {
    "pypdf": df_pypdf,
    "adobe": df_adobe_auto,
    "pdfplumber": df_pdfplumber,
    "tabula": df_tabula,
    "camelot": df_camelot,
}

A DataFrame can be constructed that contains the Levenshtein edit distance for the values in the column labels and for each column.

In [154]:
from nltk.metrics.distance import edit_distance

scores = dict()

for name, df in outputs.items():
    scores[name] = {"Column labels": edit_distance(df_pypdf.columns, df.columns)}
    
    for column_index, column_label in enumerate(df_pypdf.columns):
        scores[name][column_label] = edit_distance(
            df_pypdf.iloc[:, column_index].fillna(""),
            df.iloc[:, column_index].fillna(""),
        )

pandas.DataFrame(scores)

,pypdf,adobe,pdfplumber,tabula,camelot
Column labels,0,2,2,2,1
Name of Document/Website,0,4,2,73,0
Name of guidelines/principles,0,4,2,58,0
Issuer,0,9,1,45,0
Country of issuer,0,19,19,5,0
Type of issuer,0,30,34,44,0
Date of publishing,0,2,76,5,0
Target audience,0,6,1,42,0
Retrieval,0,1,15,58,1


However, many of the difference are due to whitespace inconsistency, _e.g._ extra and missing spaces.
If all the whitespace is removed then the result is a much closer Levenshtein edit distance.

In [187]:
scores = dict()

for name, df in outputs.items():
    scores[name] = {"Column labels": edit_distance(df_pypdf.columns.str.replace("\s+", "", regex=True), df.columns.str.replace("\s+", "", regex=True))}
    
    for column_index, column_label in enumerate(df_pypdf.columns):
        scores[name][column_label] = edit_distance(
            df_pypdf.iloc[:, column_index].fillna("").str.replace("\s+", "", regex=True),
            df.iloc[:, column_index].fillna("").str.replace("\s+", "", regex=True),
        )

pandas.DataFrame(scores)

,pypdf,adobe,pdfplumber,tabula,camelot
Column labels,0,0,0,0,1
Name of Document/Website,0,2,0,4,0
Name of guidelines/principles,0,2,0,6,0
Issuer,0,2,0,4,0
Country of issuer,0,2,0,4,0
Type of issuer,0,2,0,4,0
Date of publishing,0,2,0,4,0
Target audience,0,2,0,4,0
Retrieval,0,1,0,3,1


# Conclusions

There are over 5 trillion PDF files in the world and they hold a wealth of valuable information locked within their pages. 
Extracting tables from PDF documents can be a challenging task due to the complexity and variability of the PDF file format.
Having spend considerable effort in creating a custom table extractor using `pypdf` in [__Part 1__](RateLimiting_CompaniesHouse_Part1.ipynb) of this article, 
and here in [__Part 2__](RateLimiting_CompaniesHouse_Part2.ipynb) we considered the state-of-the-art in python packages that are designed to streamline tabular data extraction.

__Adobe__'s own API delivers table extraction and offers 500 requests per month at the free tier.
There is the overhead of setting up the API and the requirement to transmit the PDF file to Adobe for processing but Adobe successfully delivered an excel file containing the table.
__pdfplumber__ was the most straightforward package to use and delivered excellent results.
__tabula__ had the complication of setting up a link between python and a Java backend. tabula delivered the data but this had the most errors and initially the data was spread across 250 additional rows before being combined.
__camelot__ had the complication of installing ghostscript but delivered excellent results.


It is important to note that rendering PDF documents as images followed by Optical Character Recognition (OCR) is a viable alternative for table extraction. There are many online tools to help this process and recent advances in computer vision make this a powerful approach that may even replace direct PDF extraction methods in certain scenarios.

Overall, the choice of tool for extracting tables from PDFs should be guided by the specific requirements of the task.
The range of formatting in PDFs means that some customisation will be required for the best results in every solution. 
For example, here most DataFrames needed some cleaning.
Overall, there are many excellent python packages to help extract your data from PDF files.